In [72]:
from datetime import datetime
import os
import numpy as np
import pandas as pd
from als import ALSModel
from utils import validation, train
import pickle
import implicit
import scipy 

np.random.seed(42)

In [3]:
# Referenced https://github.com/storieswithsiva/Movie-Recommendation-Netflix/blob/master/Uploading%20Movie%20Recommendation%20Netflix.ipynb

In [4]:
start = datetime.now()
if not os.path.isfile('../data/archive/data.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in netflix and store them in one big file('data.csv')
    # reading from each of the four files and appending each rating to a global file 'train.csv'
    data = open('../data/archive/data.csv', mode='w')
    
    row = list()
    files=['../data/archive/combined_data_1.txt','data/archive/combined_data_2.txt', 
           '../data/archive/combined_data_3.txt', 'data/archive/combined_data_4.txt']
    for file in files:
        print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f: 
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")
    data.close()
print('Time taken :', datetime.now() - start)

Time taken : 0:00:00.000407


In [5]:
print("creating the dataframe from data.csv file..")
df = pd.read_csv('../data/archive/data.csv', sep=',', 
                       names=['movie', 'user','rating','date'])
df.date = pd.to_datetime(df.date)
print('Done.\n')

creating the dataframe from data.csv file..
Done.



In [6]:
df.head()

,movie,user,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [7]:
df.columns

Index(['movie', 'user', 'rating', 'date'], dtype='object')

In [8]:
df['id'] = df.index
df

,movie,user,rating,date,id
0,1,1488844,3,2005-09-06,0
1,1,822109,5,2005-05-13,1
2,1,885013,4,2005-10-19,2
3,1,30878,4,2005-12-26,3
4,1,823519,3,2004-05-03,4
...,...,...,...,...,...
100480502,17770,1790158,4,2005-11-01,100480502
100480503,17770,1608708,3,2005-07-19,100480503
100480504,17770,234275,1,2004-08-07,100480504
100480505,17770,255278,4,2004-05-28,100480505


In [17]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",df.shape[0])
print("Total No of Users   :", len(np.unique(df.user)))
print("Total No of movies  :", len(np.unique(df.movie)))

Total data 
--------------------------------------------------

Total no of ratings : 100480507
Total No of Users   : 480189
Total No of movies  : 17770


In [9]:
# Keep only users who rated at least 20 movies
df = df.groupby("user").filter(lambda x: len(x) >= 20)
len(np.unique(df.user))

429584

In [10]:
# Create training and validation sets
# Keep 10 random ratings of each user for validation
valid = df.groupby('user').apply(lambda x: x.sample(n=10, random_state=42))

In [11]:
valid = valid.reset_index(drop=True)
valid

,movie,user,rating,date,id
0,13050,6,3,2004-09-25,72076688
1,14302,6,4,2005-08-29,78803703
2,6221,6,3,2005-08-29,34655050
3,6196,6,4,2004-09-27,34367193
4,14203,6,3,2005-12-04,78100336
...,...,...,...,...,...
4295835,16792,2649429,5,2002-06-06,94625498
4295836,12047,2649429,5,2002-05-16,65688430
4295837,14550,2649429,5,2004-10-15,80892132
4295838,607,2649429,5,2005-06-27,3243822


In [12]:
train = df[~df.id.isin(valid.id)]
train

,movie,user,rating,date,id
0,1,1488844,3,2005-09-06,0
1,1,822109,5,2005-05-13,1
2,1,885013,4,2005-10-19,2
3,1,30878,4,2005-12-26,3
4,1,823519,3,2004-05-03,4
...,...,...,...,...,...
100480502,17770,1790158,4,2005-11-01,100480502
100480503,17770,1608708,3,2005-07-19,100480503
100480504,17770,234275,1,2004-08-07,100480504
100480505,17770,255278,4,2004-05-28,100480505


In [13]:
train.to_csv('../data/archive/train.csv', index=False, header=False)
valid.to_csv('../data/archive/valid.csv', index=False, header=False)

In [37]:
train = pd.read_csv('../data/archive/train.csv', names=['movie', 'user', 'rating', 'date', 'id'])
valid = pd.read_csv('../data/archive/valid.csv', names=['movie', 'user', 'rating', 'date', 'id'])

In [41]:
valid.head()

,movie,user,rating,date,id
0,13050,6,3,2004-09-25,72076688
1,14302,6,4,2005-08-29,78803703
2,6221,6,3,2005-08-29,34655050
3,6196,6,4,2004-09-27,34367193
4,14203,6,3,2005-12-04,78100336


In [ ]:
# This is way too large to do calculations on. Let's reduce to 1000 users.

In [43]:
users = np.random.choice(valid['user'].unique(), size=1000, replace=False)

In [44]:
train[train['user'].isin(users)].to_csv('../data/archive/train_1000.csv', index=False, header=False)
valid[valid['user'].isin(users)].to_csv('../data/archive/valid_1000.csv', index=False, header=False)

In [3]:
train_data = pd.read_csv('../data/archive/train_1000.csv', names=['movie', 'user', 'rating', 'date', 'id'])
valid_data = pd.read_csv('../data/archive/valid_1000.csv', names=['movie', 'user', 'rating', 'date', 'id'])

In [4]:
valid_data

,movie,user,rating,date,id
0,30,471,3,2004-08-03,133131
1,2016,471,4,2004-10-01,10450215
2,12799,471,2,2005-08-05,70615047
3,8904,471,5,2003-09-30,49464581
4,4345,471,5,2004-06-29,23085613
...,...,...,...,...,...
9995,17215,2646104,5,2005-08-16,97344722
9996,9756,2646104,2,2005-08-16,53560703
9997,9614,2646104,5,2005-08-16,52798269
9998,11064,2646104,4,2005-08-16,60205753


In [48]:
guarantee_items = pd.concat([pd.DataFrame([[i, -1, -1, -1, -1]], columns=['movie', 'user', 'rating', 'date', 'id']) for i in np.unique(np.append(train_data['movie'].unique(), valid_data['movie'].unique()))],ignore_index=True)
guarantee_items

,movie,user,rating,date,id
0,1,-1,-1,-1,-1
1,3,-1,-1,-1,-1
2,4,-1,-1,-1,-1
3,5,-1,-1,-1,-1
4,6,-1,-1,-1,-1
...,...,...,...,...,...
10916,17766,-1,-1,-1,-1
10917,17767,-1,-1,-1,-1
10918,17768,-1,-1,-1,-1
10919,17769,-1,-1,-1,-1


In [50]:
train_data = pd.concat([train_data, guarantee_items], ignore_index=True)
train_data.head()

,movie,user,rating,date,id
0,1,466135,4,2004-07-13,57
1,1,1751103,4,2004-08-05,414
2,3,1427751,3,2003-11-06,1287
3,3,226609,5,2003-06-18,1335
4,3,2354593,2,2005-01-19,1441


In [62]:
valid_data = pd.concat([valid_data, guarantee_items], ignore_index=True)
valid_data.head()

,movie,user,rating,date,id
0,30,471,3,2004-08-03,133131
1,2016,471,4,2004-10-01,10450215
2,12799,471,2,2005-08-05,70615047
3,8904,471,5,2003-09-30,49464581
4,4345,471,5,2004-06-29,23085613


In [63]:
# Create watch matrix
watch_matrix = pd.pivot_table(data=train_data, index='user', columns='movie', values='rating', fill_value=0)
watch_matrix = watch_matrix.applymap(lambda x: 1 if x > 0 else 0)
watch_matrix = watch_matrix.drop(-1, axis=0)
watch_matrix

movie,1,3,4,5,6,8,11,16,17,18,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2637575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2638267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# Watch matrix validation
watch_matrix_validation = pd.pivot_table(data=valid_data, index='user', columns='movie', values='rating', fill_value=0)
watch_matrix_validation = watch_matrix_validation.applymap(lambda x: 1 if x > 0 else 0)
watch_matrix_validation = watch_matrix_validation.drop(-1, axis=0)
watch_matrix_validation

movie,1,3,4,5,6,8,11,16,17,18,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10808,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631082,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2637575,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2638267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
def random_classification_noise(watch_matrix, p):
    """
    watch_matrix: watch matrix of watched or not
    p: probability of disliking an interacted with item
    """
    # Create a copy of the watch matrix
    watch_matrix_noisy = watch_matrix.copy()
    
    # For each user, randomly select p% of their interacted with items and set them to 0
    watch_matrix_noisy = watch_matrix_noisy.applymap(lambda x: -1 if x == 1 and np.random.random() < p else x)

    return watch_matrix_noisy.values

In [66]:
# Generate preference matrix based on simulation for validation dataset.
# Start training for 20 epochs

In [67]:
random_classification_noise(watch_matrix_validation, 0.1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
watch_matrix_validation.values.shape

(1000, 10921)

In [69]:
watch_matrix.values.shape

(1000, 10921)

In [73]:
watch_matrix_csr = scipy.sparse.csr_matrix(watch_matrix.values)

In [ ]:
# Modify source code to have validation acc. Change callback to also pass in the entire object.

In [82]:
def validation_hook(epoch, elapsed_time, progress):
    print("===\nEpoch: ", epoch)
    print("Training Loss: " + progress)

    # Calculate validation loss
    

SyntaxError: invalid syntax. Perhaps you forgot a comma? (391461771.py, line 3)

In [81]:
model = implicit.als.AlternatingLeastSquares(factors=100, calculate_training_loss=True, iterations=20)
model.fit(watch_matrix_csr, callback=validation_hook)

  5%|▌         | 1/20 [00:01<00:16,  1.14it/s, loss=0.0101]

0 1.1733860969543457 0.01006169337409946


 10%|█         | 2/20 [00:02<00:17,  1.03it/s, loss=0.00798]

1 1.0412070751190186 0.00798209919965246


 15%|█▌        | 3/20 [00:03<00:17,  1.02s/it, loss=0.00748]

...

17 1.143847942352295 0.007041995106187522


 95%|█████████▌| 19/20 [00:20<00:01,  1.12s/it, loss=0.00704]

18 1.1536221504211426 0.007040934405680113


100%|██████████| 20/20 [00:22<00:00,  1.10s/it, loss=0.00704]

19 1.4113271236419678 0.007040021742960168


In [70]:
res = []
for p in [0.25, 0.5, 0.75]:
    true_pref_validation = random_classification_noise(watch_matrix_validation, p)
    model = ALSModel(watch_matrix.values)
    model, training_losses, proxy_losses, true_losses = train(model, watch_matrix.values, watch_matrix_validation.values, true_pref_validation, num_epochs=20)
    res.append((p, training_losses, proxy_losses, true_losses))

with open('../data/archive/results.txt', 'w') as f:
    pickle.dump(res, f)



===
Epoch:  0


KeyboardInterrupt: 